<a href="https://colab.research.google.com/github/2403a52030-sketch/Lab-Assignment-Codes/blob/main/Assignment_18.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task-1:** **Movie Database API**

Connect to a Movie Database API (e.g., OMDb or TMDB) to
fetch details of a movie

**Prompt:**

Generate Python code that connects to a Movie Database API (such as OMDb or TMDB) to fetch details of a movie by its title.
The program should:

Accept a movie title as user input.

Query the API using the provided title.

Handle possible errors, including invalid movie name, missing or expired API key, and request timeout.

Display the movie’s Title, Release Year, Genre, IMDb Rating, and Director in a clean output format.

Include comments explaining each part of the code

In [1]:
import requests
import os

# Function to get the API key from Colab secrets
def get_api_key():
    """Retrieves the API key from Colab secrets."""
    # Replace 'TMDB_API_KEY' with the name you used in Colab secrets
    api_key = os.environ.get('TMDB_API_KEY')
    if not api_key:
        raise ValueError("API key not found. Please set 'TMDB_API_KEY' in Colab secrets.")
    return api_key

# Function to fetch movie details
def fetch_movie_details(movie_title):
    """
    Fetches movie details from the TMDB API.

    Args:
        movie_title (str): The title of the movie to search for.

    Returns:
        dict: A dictionary containing movie details, or None if not found or an error occurred.
    """
    try:
        api_key = get_api_key()
        base_url = "https://api.themoviedb.org/3"
        search_endpoint = f"{base_url}/search/movie"

        # Parameters for the search request
        params = {
            'api_key': api_key,
            'query': movie_title
        }

        # Make the search request
        response = requests.get(search_endpoint, params=params, timeout=10) # Set a timeout of 10 seconds
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        data = response.json()

        # Check if any results were found
        if data and data['results']:
            # Get the first result (assuming it's the most relevant)
            movie_id = data['results'][0]['id']

            # Now fetch detailed information for the movie
            details_endpoint = f"{base_url}/movie/{movie_id}"
            details_response = requests.get(details_endpoint, params={'api_key': api_key}, timeout=10)
            details_response.raise_for_status()
            movie_details = details_response.json()

            return movie_details
        else:
            print(f"Movie '{movie_title}' not found.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching movie details: {e}")
        return None
    except ValueError as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Function to display movie details
def display_movie_details(movie_details):
    """
    Displays the movie details in a clean format.

    Args:
        movie_details (dict): A dictionary containing movie details.
    """
    if movie_details:
        print("\n--- Movie Details ---")
        print(f"Title: {movie_details.get('title', 'N/A')}")
        print(f"Release Year: {movie_details.get('release_date', 'N/A').split('-')[0] if movie_details.get('release_date') else 'N/A'}")
        # Extract genre names from the list of genre dictionaries
        genres = [genre['name'] for genre in movie_details.get('genres', [])]
        print(f"Genre: {', '.join(genres) if genres else 'N/A'}")
        # TMDB API does not directly provide IMDb rating, you would need to use another API or a different endpoint if available
        print(f"IMDb Rating: N/A (Not directly available from this TMDB endpoint)")
        # Director information is not directly available in this TMDB endpoint, you would need to fetch credits
        print(f"Director: N/A (Not directly available from this TMDB endpoint)")
        print("-----------------------")

# Main part of the program
if __name__ == "__main__":
    # Get movie title from user input
    movie_title = input("Enter the movie title: ")

    # Fetch and display movie details
    details = fetch_movie_details(movie_title)
    if details:
        display_movie_details(details)

Enter the movie title: inception
Error: API key not found. Please set 'TMDB_API_KEY' in Colab secrets.


**Explaination:**

**Import necessary libraries:**

requests: This library is used to make HTTP requests to the TMDB API.

os: This library is used to access environment variables, which is how the code retrieves the API key stored in Colab secrets.

**get_api_key() function:**

This function is responsible for securely retrieving your TMDB API key from Colab's secrets manager.

It looks for an environment variable named TMDB_API_KEY. You need to store your API key in Colab secrets with this exact name.

If the key is not found, it raises a ValueError instructing the user to set the API key in Colab secrets.

**fetch_movie_details(movie_title) function:**

This is the core function that interacts with the TMDB API.

It takes the movie_title as input.

It constructs the base URL and the search endpoint for the TMDB API.

It sets up the params dictionary containing the api_key and the query (movie title) for the API request.

It makes a GET request to the search endpoint using requests.get(), including a timeout to prevent the request from hanging indefinitely.

response.raise_for_status() checks if the request was successful. If there's a bad status code (like 404 or 500), it raises an HTTPError.

It parses the JSON response from the API.

It checks if the search returned any results. If so, it takes the id of the first result (assuming it's the most relevant match).

Using the movie id, it makes another GET request to the movie details endpoint to fetch more comprehensive information about the movie.

It returns the movie_details dictionary if the details are successfully fetched, otherwise returns None.

It includes try...except blocks to handle potential errors like network issues (requests.exceptions.RequestException), missing API key (ValueError), or other unexpected errors.

**Task-2:** **Public Transport API**

Use a Public Transport API (e.g., city bus/train API or mock
data) to fetch live arrival times

**Prompt:**

Generate Python code to fetch the next 5 arrivals from a Public Transport API using a given station ID.
Handle invalid station codes, no service, and bad responses.
Display a table showing Route Number, Destination, and Arrival Time

In [2]:
import pandas as pd
import time # Import time for simulating delays

# Mock data to simulate API response
MOCK_ARRIVAL_DATA = {
    "station_A": [
        {"route": "101", "destination": "Downtown", "arrival_time": int(time.time()) + 300}, # 5 minutes from now
        {"route": "202", "destination": "Uptown", "arrival_time": int(time.time()) + 600}, # 10 minutes from now
        {"route": "101", "destination": "Downtown", "arrival_time": int(time.time()) + 900}, # 15 minutes from now
        {"route": "303", "destination": "Suburbia", "arrival_time": int(time.time()) + 1200}, # 20 minutes from now
        {"route": "202", "destination": "Uptown", "arrival_time": int(time.time()) + 1500}, # 25 minutes from now
        {"route": "101", "destination": "Downtown", "arrival_time": int(time.time()) + 1800}, # 30 minutes from now
    ],
    "station_B": [
        {"route": "404", "destination": "Airport", "arrival_time": int(time.time()) + 180}, # 3 minutes from now
        {"route": "505", "destination": "City Center", "arrival_time": int(time.time()) + 720}, # 12 minutes from now
        {"route": "404", "destination": "Airport", "arrival_time": int(time.time()) + 1080}, # 18 minutes from now
    ],
    "station_C": [] # Simulate a station with no upcoming arrivals
}

# Function to fetch live arrival times (simulated API call)
def fetch_arrival_times(station_id):
    """
    Simulates fetching live arrival times for a given station ID.

    Args:
        station_id (str): The ID of the public transport stop or station.

    Returns:
        list: A list of dictionaries containing arrival details, or None if an error occurs.
    """
    print(f"Fetching arrival times for station: {station_id}...")
    # Simulate API call delay
    time.sleep(1)

    # Simulate error handling
    if station_id not in MOCK_ARRIVAL_DATA:
        print(f"Error: Invalid station ID '{station_id}'.")
        return None
    elif not MOCK_ARRIVAL_DATA[station_id]:
        print(f"No upcoming arrivals for station '{station_id}'.")
        return []
    else:
        # Simulate successful API response
        return MOCK_ARRIVAL_DATA[station_id]

# Function to display arrival times in a table
def display_arrival_times(arrival_data):
    """
    Displays arrival times in a readable table format.

    Args:
        arrival_data (list): A list of dictionaries containing arrival details.
    """
    if arrival_data is None:
        return # Handle the case where fetching failed

    if not arrival_data:
        print("No arrival data to display.")
        return

    # Convert arrival times from timestamps to readable format
    formatted_data = []
    for arrival in arrival_data:
        arrival_time_readable = time.strftime('%H:%M:%S', time.localtime(arrival['arrival_time']))
        formatted_data.append({
            "Route Number": arrival["route"],
            "Destination": arrival["destination"],
            "Arrival Time": arrival_time_readable
        })

    # Display the data using pandas DataFrame for a nice table format
    df = pd.DataFrame(formatted_data)
    print("\n--- Upcoming Arrivals ---")
    # Display only the next 5 arrivals
    display(df.head(5))
    print("-------------------------")

# Main part of the program
if __name__ == "__main__":
    # Get station ID from user input
    station_id = input("Enter the station ID (e.g., station_A, station_B, station_C): ")

    # Fetch and display arrival times
    arrival_details = fetch_arrival_times(station_id)
    display_arrival_times(arrival_details)

Enter the station ID (e.g., station_A, station_B, station_C): station_a
Fetching arrival times for station: station_a...
Error: Invalid station ID 'station_a'.


**Explaination:**

**Import necessary libraries:**

pandas: Used to create and display the arrival data in a formatted table (DataFrame).

time: Used to get the current time (timestamp) and simulate delays in the API call.

**MOCK_ARRIVAL_DATA:**

This is a Python dictionary that simulates the response you would get from a real public transport API.

The keys of the dictionary represent different station_ids (e.g., "station_A", "station_B").

The values are lists of dictionaries, where each inner dictionary represents an upcoming arrival with details like route, destination, and arrival_time (as a timestamp).

station_C is included to simulate a station with no upcoming arrivals.

**fetch_arrival_times(station_id) function:**

This function simulates fetching live arrival times for a given station_id.
It prints a message indicating that it's fetching data.

time.sleep(1) simulates the delay that would occur when making a real API call over the network.

**Task-3:** Stock Market/Financial Data API

Connect to a stock data API (e.g., Alpha Vantage, Yahoo
Finance) to fetch daily stock prices

Prompt:
Generate Python code that connects to a Stock Data API (e.g., Alpha Vantage or Yahoo Finance) to fetch daily stock prices by ticker symbol.
Handle API call limits, invalid ticker symbols, and null responses.
Display the Opening Price, Closing Price, High, Low, and Trading Volume

In [3]:
import requests
import os
import pandas as pd

# Function to get the API key from Colab secrets
def get_api_key():
    """Retrieves the API key from Colab secrets."""
    # Replace 'ALPHA_VANTAGE_API_KEY' with the name you used in Colab secrets
    api_key = os.environ.get('ALPHA_VANTAGE_API_KEY')
    if not api_key:
        raise ValueError("API key not found. Please set 'ALPHA_VANTAGE_API_KEY' in Colab secrets.")
    return api_key

# Function to fetch daily stock prices
def fetch_daily_stock_prices(ticker_symbol):
    """
    Fetches daily stock prices from the Alpha Vantage API.

    Args:
        ticker_symbol (str): The ticker symbol of the stock.

    Returns:
        dict: A dictionary containing daily stock data, or None if an error occurred or no data found.
    """
    try:
        api_key = get_api_key()
        base_url = "https://www.alphavantage.co/query"

        # Parameters for the API request
        params = {
            'function': 'TIME_SERIES_DAILY',
            'symbol': ticker_symbol,
            'apikey': api_key,
            'outputsize': 'compact' # Get the latest 100 data points
        }

        # Make the API request
        response = requests.get(base_url, params=params, timeout=10) # Set a timeout of 10 seconds
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        data = response.json()

        # Check for API call limits or invalid ticker
        if "Error Message" in data:
            print(f"Error from API: {data['Error Message']}")
            return None
        if "Note" in data:
            print(f"API Note: {data['Note']}")
            # This might indicate hitting the API limit, but we'll still try to process
            pass # Or you could return None here depending on desired behavior
        if not data or 'Time Series (Daily)' not in data:
            print(f"No daily time series data found for ticker symbol '{ticker_symbol}'.")
            return None

        return data['Time Series (Daily)']

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock data: {e}")
        return None
    except ValueError as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Function to display stock data
def display_stock_data(stock_data):
    """
    Displays daily stock data in a readable format.

    Args:
        stock_data (dict): A dictionary containing daily stock data.
    """
    if not stock_data:
        print("No stock data to display.")
        return

    # Convert the dictionary to a pandas DataFrame for better display
    df = pd.DataFrame.from_dict(stock_data, orient='index')

    # Rename columns for clarity
    df.columns = ['Opening Price', 'High', 'Low', 'Closing Price', 'Volume']

    # Convert columns to numeric, coercing errors
    for col in ['Opening Price', 'High', 'Low', 'Closing Price', 'Volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Display the latest data point (today's or the most recent trading day)
    print("\n--- Latest Stock Data ---")
    # Sort by index (date) in descending order to get the latest first
    df.sort_index(ascending=False, inplace=True)
    display(df.head(1))
    print("------------------------")

# Main part of the program
if __name__ == "__main__":
    # Get ticker symbol from user input
    ticker_symbol = input("Enter the stock ticker symbol (e.g., AAPL, GOOGL): ").upper()

    # Fetch and display stock data
    daily_data = fetch_daily_stock_prices(ticker_symbol)
    if daily_data:
        display_stock_data(daily_data)

Enter the stock ticker symbol (e.g., AAPL, GOOGL): AAPL
Error: API key not found. Please set 'ALPHA_VANTAGE_API_KEY' in Colab secrets.


Explaination:

**Import necessary libraries:**

requests: This library is used to make HTTP requests to the Alpha Vantage API.

os: This library is used to access environment variables, which is how the code retrieves the API key stored in Colab secrets.

pandas: Used to organize and display the fetched stock data in a structured DataFrame format.

**get_api_key() function:**

This function is responsible for securely retrieving your Alpha Vantage API key from Colab's secrets manager.

It looks for an environment variable named ALPHA_VANTAGE_API_KEY. You need to store your API key in Colab secrets with this exact name.

If the key is not found, it raises a ValueError instructing the user to set the API key in Colab secrets.

Task-4:  Real-Time Application: Translation API

Scenario: Build a translator using a free Translation API (e.g.,
Libre Translate, Google Translate)

Prompt:

Generate Python code using a free Translation API (like Libre Translate or Google Translate) to translate text.  
Take input text and target language from the user, handle invalid language codes, empty input, and API errors.  
Include a retry mechanism and display both original and translated text clearly.

In [5]:
import requests
import time

# LibreTranslate API endpoint (using a public instance - replace with your own if hosting)
API_URL = "https://translate.argosopentech.com/translate" # Example public instance

# Function to translate text with retry mechanism
def translate_text_with_retry(text, target_language, source_language="auto", retries=3, delay=2):
    """
    Translates text using the LibreTranslate API with a retry mechanism.

    Args:
        text (str): The text to translate.
        target_language (str): The target language code (e.g., 'en', 'es', 'fr').
        source_language (str): The source language code, 'auto' for auto-detection.
        retries (int): The number of times to retry the translation if it fails.
        delay (int): The delay in seconds between retries.

    Returns:
        str: The translated text, or None if translation fails after retries.
    """
    if not text.strip():
        print("Error: Input text is empty.")
        return None

    for attempt in range(retries + 1):
        print(f"Attempt {attempt + 1} of {retries + 1} to translate...")
        try:
            payload = {
                "q": text,
                "source": source_language,
                "target": target_language
                # Add "api_key": "YOUR_API_KEY" if using an instance that requires one
            }
            response = requests.post(API_URL, json=payload, timeout=10) # Set a timeout
            response.raise_for_status() # Raise an exception for bad status codes

            data = response.json()

            if "error" in data:
                print(f"API error: {data['error']}")
                if attempt < retries:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                else:
                    print("Max retries reached. Translation failed.")
                    return None
            elif "translatedText" in data:
                return data["translatedText"]
            else:
                print("Unexpected API response format.")
                if attempt < retries:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                else:
                    print("Max retries reached. Translation failed.")
                    return None

        except requests.exceptions.RequestException as e:
            print(f"Request failed on attempt {attempt + 1}: {e}")
            if attempt < retries:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Translation failed.")
                return None
        except Exception as e:
            print(f"An unexpected error occurred on attempt {attempt + 1}: {e}")
            if attempt < retries:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Translation failed.")
                return None


# Main part of the program
if __name__ == "__main__":
    print("--- Simple Translator ---")
    print("Note: Using a public LibreTranslate instance. May have usage limits.")
    while True:
        # Get input text from user
        input_text = input("Enter text to translate (or type 'quit' to exit): ")
        if input_text.lower() == 'quit':
            break

        # Get target language from user
        # You might want to provide a list of supported language codes
        target_lang = input("Enter target language code (e.g., en, es, fr): ").lower()

        # Translate the text
        translated_text = translate_text_with_retry(input_text, target_lang)

        # Display results
        if translated_text is not None:
            print("\n--- Result ---")
            print(f"Original Text: {input_text}")
            print(f"Translated Text: {translated_text}")
            print("--------------")
        else:
            print("\nTranslation could not be completed.")

        print("-" * 20) # Separator for next translation

--- Simple Translator ---
Note: Using a public LibreTranslate instance. May have usage limits.
Enter text to translate (or type 'quit' to exit): quit


Explaination:

Import necessary libraries:

requests: This library is used to make HTTP requests to the LibreTranslate API.

time: This library is used for pausing the program execution, specifically for implementing the retry delay.

**API_URL:**

This variable holds the endpoint URL for the LibreTranslate API. In this code,
